# Lecture des échantillons 


In [11]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it

In [12]:
debug=False

In [13]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [14]:
listeTirages=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage.pkl")
listeVHF=[u"être","avoir","faire","dire","pouvoir","aller","voir","savoir","vouloir","venir"]

In [15]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [16]:
def nbFormesEchantillons(nom):
    print nom
    for tirage in tiragesNom(nom):
        sample=lireLexique(tirage)
        print sample[sample["tir1"]>0]["tir1"].count(),
        print sample[sample["tir1"]>0]["tir1"].sum(),
        print tirage

In [17]:
listeTirages200=tiragesNom("200Mo")
listeTirages20=tiragesNom("20Mo")
#len(listeTirages200)
listeTirages1=tiragesNom("1Mo")
listeTirages50k=tiragesNom("50Ko")
listeTirages100k=tiragesNom("100Ko")


tailleLexique=[]
nomLexique0=listeTirages200.pop(0)
nomLexique0=listeTirages1.pop(0)
lexique=lireLexique(nomLexique0)
taille=lexique[lexique["tir1"]>0]["tir1"].count()
print taille, lexique[lexique["tir1"]>0]["tir1"].sum()
tailleLexique.append(taille)

In [24]:
def reduceTrad(el,traductions):
    if traductions[el]==el:
        return el
    else:
        return reduceTrad(traductions[el],traductions)
    
def defParadigme(paradigmes):
    syncretisms=[]
    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    for (c1,c2) in it.combinations(cases, 2):
        c1Val=paradigmes[c1].notnull()
        c2Val=paradigmes[c2].notnull()
        c1Sur=paradigmes[c1].str.contains(",")
        c2Sur=paradigmes[c2].str.contains(",")
        l1=len(paradigmes[paradigmes[c1].notnull()])
        l2=len(paradigmes[paradigmes[c2].notnull()])
        paire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
        lenDiff=len(paire[~paire[c1].str.contains(",") & ~paire[c2].str.contains(",")])
        if lenDiff>0:
            if debug:
                print u"%s ≠ %s"%(c1,c2)
                print "différence",lenDiff
                if lenDiff<12:
                    print paire
        else:
            surAbondant=paire[paire[c1].str.contains(",") | paire[c2].str.contains(",")]
#            print "--------------------------------"
            if len(surAbondant)==0:
#                print u"%s = %s"%(c1,c2) 
                syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                compatible=True
                for index,row in surAbondant.iterrows():
                    if "," in row[c1]:
                        if "," in row[c2]:
                            if row[c1]!=row[c2]:
                                compatible=False
                        else:
                            if not row[c2] in row[c1].split(","):
                                compatible=False
                    else:
                        if not row[c1] in row[c2].split(","):
                            compatible=False
                if compatible:
#                    print u"%s = %s"%(c1,c2)
                    syncretisms.append(u"%s = %s"%(c1,c2))
                else:
                    print u"%s ≠ %s"%(c1,c2)
                    print surAbondant

    reductionParadigme={c:c for c in cases}
    for syncretism in syncretisms:
        c1,c2=syncretism.split(" = ")
        removeC=max(c1,c2)
        keyC=min(c1,c2)
        if removeC in reductionParadigme:
            if keyC in reductionParadigme:
                key=min(reductionParadigme[keyC],reductionParadigme[removeC])
            else:
                key=min(keyC,reductionParadigme[removeC])
        else:
            if keyC in reductionParadigme:
                key=reductionParadigme[keyC]
            else:
                key=keyC
        reductionParadigme[removeC]=key
        reductionParadigme[keyC]=key

    for el in reductionParadigme:
        reductionParadigme[el]=reduceTrad(el,reductionParadigme)

    paradigmeReduit = {}
    for k, v in reductionParadigme.iteritems():
        paradigmeReduit[v] = paradigmeReduit.get(v, [])
        paradigmeReduit[v].append(k)
    print "syncrétismes",len(paradigmeReduit), sorted(paradigmeReduit.keys())
    print paradigmeReduit

In [19]:
def pivotLexique(lexique):
    paradigmes=pd.pivot_table(lexique[lexique["tir1"]>0], values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    defParadigme(paradigmes)

In [25]:
for nomLexique in sorted(listeTirages100k):
    lexique=lireLexique(nomLexique)
    taille=lexique[lexique["tir1"]>0]["tir1"].count()
    print nomLexique
    print taille, lexique[lexique["tir1"]>0]["tir1"].sum()
    pivotLexique(lexique)

/Users/gilles/Box Sync/2015-Data/MGC-170330-00-100Ko-Tirage.pkl
2444 5000
syncrétismes 4 [u'ai1P', u'inf', u'pI2P', u'pI2S']
{u'ai1P': [u'ii1P', u'ii1S', u'pP', u'ai3S', u'ai3P', u'is3S', u'ps3S', u'ii3S', u'ai1P', u'ii3P', u'pi2S', u'ai1S', u'pi2P', u'ppFS', u'ppFP', u'ps1P', u'ps3P', u'ps1S', u'fi2P', u'fi2S', u'ps2P', u'ps2S', u'ai2S', u'pc2P', u'pc2S', u'ii2P', u'pi1S', u'pi1P', u'ii2S', u'pi3P', u'pi3S', u'ppMS', u'pc3S', u'pc3P', u'pc1P', u'pc1S', u'fi3S', u'fi3P', u'ppMP', u'fi1P', u'fi1S', u'pI1P'], u'inf': [u'inf'], u'pI2S': [u'pI2S'], u'pI2P': [u'pI2P']}
/Users/gilles/Box Sync/2015-Data/MGC-170330-01-100Ko-Tirage.pkl
4049 10000
syncrétismes 10 [u'ai1P', u'fi2P', u'inf', u'pI2P', u'pI2S', u'pi1P', u'pi1S', u'pi2S', u'pi3P', u'pi3S']
{u'pi1P': [u'pi1P'], u'pI2S': [u'pI2S'], u'pi1S': [u'pi1S'], u'ai1P': [u'ii1P', u'ii1S', u'pP', u'ai3S', u'ai3P', u'is3P', u'is3S', u'ps3S', u'ii3S', u'ai1P', u'ii3P', u'ai1S', u'pi2P', u'ppFS', u'ppFP', u'ps1P', u'ps3P', u'ps1S', u'fi2S', u'ps2P',